## Preambule

In [1]:
import numpy as np
import xarray as xr
from importlib import reload
import pandas as pd
from tqdm import tqdm
import json
from pathlib import Path
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
import Functions
reload(Functions)
from Functions import (pop_func, emis_func, determine_coefficient,
                       popshare_func, emisshare_func, emis_total_func, rho, cumpopshare_func,
                       cumfuturepop_func, emis_f_func, gdp_future, create_groups, gdp_future_reread, grouping, grouping_xarray)

## Run class or read xarray file

In [19]:
import ClassEffortSharing
reload(ClassEffortSharing)
from ClassEffortSharing import shareefforts

efshare = shareefforts()
efshare.define_paths()
efshare.read_countrygroups()
if efshare.reread == True:
    efshare.read_unpop()
    efshare.read_hdi()
    efshare.read_gdp()
    efshare.read_edgar()
    efshare.read_primap()
    efshare.read_ar6()
    efshare.read_rci()
    efshare.read_ndc_cr()
    efshare.read_ndc()
    efshare.read_baseline()
    efshare.concat_xrobject()
    efshare.save_xrs_databases()
else:
    efshare.load_xrs_databases()
efshare.general_calculations()
efshare.save_budgets()
self = efshare

# ==================================== #
# Initializing shareefforts class    #
# ==================================== #
- Defining paths
- Reading country groups
- Load XR databases
- General calculations for effort sharing
- Save budgets
